In [1]:
import pickle
import pandas as pd
from pathlib import Path

table_name = "comments_melting_point.pickle"
data_dir = Path("../Data")
file_path = data_dir / table_name

with open(file_path, "rb") as f:
    data = pickle.load(f)

df = pd.DataFrame(data)

df.head()

,0,1,2,3
0,000010011,438 K.,164.85,164.85
1,000010028,348 K.,74.85,74.85
2,000010039,565 K.,291.85,291.85
3,000010050,597 K.,323.85,323.85
4,000010069,557 K.,283.85,283.85


In [2]:
df.tail()

,0,1,2,3
158071,050105287,556 K,282.85,282.85
158072,050105333,369 K,95.85,95.85
158073,050105335,433 K,159.85,159.85
158074,050105336,433 K,159.85,159.85
158075,050105350,166 K,-107.15,-107.15


In [3]:
166 - 273 #--> it seems the lower and higher limits are just melting point in degree Celcius 

-107

In [4]:
new_colomn_names = ["ProductID", "Melting Points", "Lower Limit", "Higher Limit"]

column_maps = {i:n for i, n in enumerate(new_colomn_names)}

df_renamed = df.rename(column_maps, axis=1)

In [5]:
df_renamed

,ProductID,Melting Points,Lower Limit,Higher Limit
0,000010011,438 K.,164.85,164.85
1,000010028,348 K.,74.85,74.85
2,000010039,565 K.,291.85,291.85
3,000010050,597 K.,323.85,323.85
4,000010069,557 K.,283.85,283.85
...,...,...,...,...
158071,050105287,556 K,282.85,282.85
158072,050105333,369 K,95.85,95.85
158073,050105335,433 K,159.85,159.85
158074,050105336,433 K,159.85,159.85


In [6]:
df_renamed = df_renamed[["ProductID", "Melting Points"]]

In [7]:
df_renamed.head()

,ProductID,Melting Points
0,000010011,438 K.
1,000010028,348 K.
2,000010039,565 K.
3,000010050,597 K.
4,000010069,557 K.


In [8]:
# check if all melting points are in K
df_renamed["Melting Points"].str.contains("K").count()

158076

In [9]:
# # 1. Remove " K.", "K", etc.
# df_clean = df_renamed["Melting Points"].str.replace(r"\s*K\.?|K", "", regex=True)

# # 2. Remove extra characters if needed (e.g., °, °C, comments, etc.)
# df_clean = df_clean.str.extract(r"(\d+)", expand=False)  # extract the first number only

# # 3. Convert to integers (or float if decimals exist)
# df_clean = pd.to_numeric(df_clean, errors='coerce').astype('float')  # nullable integer type


In [10]:
df_renamed["Melting Points"] = df_renamed["Melting Points"].str.replace(r"\s*K\.?|K", "", regex=True).str.extract(r"(\d+)", expand=False).astype('float')

/var/folders/p1/_xzph12174z0rlnng642dchm0000gq/T/ipykernel_15086/2584377211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_renamed["Melting Points"] = df_renamed["Melting Points"].str.replace(r"\s*K\.?|K", "", regex=True).str.extract(r"(\d+)", expand=False).astype('float')


In [11]:
df_renamed

,ProductID,Melting Points
0,000010011,438.0
1,000010028,348.0
2,000010039,565.0
3,000010050,597.0
4,000010069,557.0
...,...,...
158071,050105287,556.0
158072,050105333,369.0
158073,050105335,433.0
158074,050105336,433.0


In [12]:
file_name = "melting-points.csv"

file_path = data_dir / file_name

df_renamed.to_csv(file_path, index=False)


### Screening based on application

In [24]:
# read application file

application_df = pd.read_csv(data_dir / "applications.csv", dtype={'ProductID': str})
application_df.head()

,ProductID,Description
0,000010359,Battery Material
1,000010359,Intercalate
2,000010503,Ionic Conductor
3,000010579,Superconducting Material
4,000010579,Battery Material


In [28]:
product_ids = application_df["ProductID"].to_list()

print(df_renamed[df_renamed.ProductID.isin(product_ids)])



        ProductID  Melting Points
45      000010359          1073.0
111     000010579           585.0
139     000010646          3800.0
150     000010677          1918.0
166     000010706          1068.0
...           ...             ...
149159  040289696          1046.0
149160  040289750           997.0
149191  050050013           520.0
149421  050050785           407.0
151025  050057108           482.0

[525 rows x 2 columns]


In [29]:
df_renamed_selected = df_renamed[df_renamed.ProductID.isin(product_ids)]

In [31]:
file_name = "melting-points-energy.csv"

file_path = data_dir / file_name

df_renamed_selected.to_csv(file_path, index=False)